In [1]:
import tweepy
import os
import json
from IPython.display import JSON
import logging
import parser
import traceback
import sys
import sqlalchemy

Oauth handling (probably should do a class later). Shouldalso use an generator for secret and key but as twitter doesn't expire it it will do for the moment. 
In a production version I would probably generate an app by account so I can track each account usage. 

Oauth 2 could also psossibly be a solution as I only need public infos. 

In [2]:
consumer_key = os.environ['TWITTER_API_KEY']
consumer_secret = os.environ['TWITTER_API_SECRET']
key = os.environ['TWITTER_ACCESS_TOKEN']
secret = os.environ['TWITTER_ACCESS_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(key, secret)
api = tweepy.API(auth)

# using tweepy 
The main objective is to gather infos from twitter. We're gonna use the stream endpoint because using the search doesn't return all the tweets.

Probably end up bumping into api limitations pretty fast but it will be ok for the demo.

The process will be the following : 

- Get latest tweet 
- Store Raw response (probably in an S3/minio bucket. will probably do it in second time but it should be a good idea in case the processing crashes
- get relevant datas
- run additional infos gathering and transform 
      - detect mood ? 
      - lang detection ? 
      - ...
- store in the sql base (shared) 

Airflow process : 
- Each hour detect new keywords. 
- For each keyord create a notebook (aka a thread) 
- Check if all notebook Running. 
- can mark a keyword as inactive in Flask then  don't check 


In [3]:
#set up our main class using tweepy.StreamListener
class TwitterListener(tweepy.StreamListener):
    
    def __init__(self):
        self.database = os.environ['SHARED_DB_URI']

    def on_error(self, status_code):
        if status_code == 420:
            # returning False in on_data disconnects the stream
            return False

    def on_status(self, status):
        print(status.text)
        return True

    def on_data(self, data):
        """
        Automatic detection of the kind of data collected from Twitter
        This method reads in tweet data as JSON and extracts the data we want.
        """
        try:
            # parse as json
            raw_data = json.loads(data)

            # extract the relevant data
            if "text" in raw_data:
                user = raw_data["user"]["screen_name"]
                created_at = raw_data["created_at"]
                tweet = raw_data["text"]
                retweet_count = raw_data["retweet_count"]
                id_str = raw_data["id_str"]

            #insert data just collected into MySQL my_database
            #populate_table(user, created_at, tweet, retweet_count, id_str)
            print(f"Tweet colleted at: {created_at}")

        except Exception as e:
            logging.error(traceback.format_exc())
            # Logs the error appropriately. 
            

# todo : create the tables on init
    def populate_table(
        self, user, created_at, tweet, retweet_count, id_str, raw_data
    ):
        """Populate a given table witht he Twitter collected data

        Args:
            user (str): username from the status
            created_at (datetime): when the tweet was created
            tweet (str): text
            retweet_count (int): number of retweets
            id_str (int): unique id for the tweet
            raw_data (json) : storing raw data for further usage
        """

        dbconnect = connect_db(self.database)

        cursor = dbconnect.cursor()
        cursor.execute("USE airflowdb")

        # add content here

        try:
            # what is missing? 
            commit()
            print("commited")

        except mysql.Error as e:
            print(e)
            dbconnect.rollback()

        cursor.close()
        dbconnect.close()

        return


In [ ]:
tweepy_listener = TwitterListener()
tweepy_stream = tweepy.Stream(auth = api.auth, listener=tweepy_listener)
results = tweepy_stream.filter(track=['Minecraft'])

Tweet colleted at: Thu Jan 09 23:52:13 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:13 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:19 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:19 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:20 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:23 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:23 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:38 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:38 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:38 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:41 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:43 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:46 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:52 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:53 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:54 +0000 2020
Tweet colleted at: Thu Jan 09 23:52:57 +0000 2020


In [ ]:
print(results)